# Distributed Training with TransformersTrainer

This notebook demonstrates how to use the `TransformersTrainer` from the Kubeflow SDK to run distributed fine-tuning of HuggingFace models on Red Hat OpenShift AI.

**What you will learn:**
- How to define a training function for HuggingFace Transformers
- How to configure distributed training with TransformersTrainer
- How to submit and monitor training jobs using the Kubeflow SDK
- How to view real-time training progress

**Prerequisites:**
- Access to an OpenShift AI cluster with Kubeflow Trainer enabled
- A workbench with Python 3.9+

## 1. Install dependencies

In [2]:
%pip install "kubeflow @ git+https://github.com/opendatahub-io/kubeflow-sdk.git@v0.2.1+rhai0"

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/opendatahub-io/kubeflow-sdk.git (to revision v0.2.1+rhai0) to /private/var/folders/g7/rfmmh71x7fzbvrv5lwk93p140000gn/T/pip-install-yayrl786/kubeflow_658e3a46c55a4229a0da4e2dc5b15a09
  Running command git clone --filter=blob:none --quiet https://github.com/opendatahub-io/kubeflow-sdk.git /private/var/folders/g7/rfmmh71x7fzbvrv5lwk93p140000gn/T/pip-install-yayrl786/kubeflow_658e3a46c55a4229a0da4e2dc5b15a09
  Running command git checkout -q 7b3d014a8434af6f55897a274a6aa68af3fdf87f
  Resolved https://github.com/opendatahub-io/kubeflow-sdk.git to commit 7b3d014a8434af6f55897a274a6aa68af3fdf87f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Verify installation
import kubeflow
print(f"Kubeflow SDK version: {kubeflow.__version__}")

from kubeflow.trainer import TrainerClient
from kubeflow.trainer.rhai import TransformersTrainer
from kubeflow.common.types import KubernetesBackendConfig

print("SDK imported successfully")

Kubeflow SDK version: v0.2.1+rhai0


/Users/knema/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


SDK imported successfully


## 2. Configuration

Modify these values based on your cluster resources.

In [4]:
# Training configuration
NAMESPACE = None  # None = use current namespace from kubeconfig

# Distributed training settings
NUM_NODES = 2
GPUS_PER_NODE = 1

print(f"Configuration:")
print(f"  Nodes: {NUM_NODES}")
print(f"  GPUs per node: {GPUS_PER_NODE}")
print(f"  Total GPUs: {NUM_NODES * GPUS_PER_NODE}")

Configuration:
  Nodes: 2
  GPUs per node: 1
  Total GPUs: 2


## 3. Define the training function

This function will be executed on each training node. All imports must be inside the function.

In [5]:
def train_func():
    """Distributed training function for IMDB sentiment classification."""
    import os
    import torch
    from transformers import (
        AutoModelForSequenceClassification,
        AutoTokenizer,
        Trainer,
        TrainingArguments,
    )
    from datasets import load_dataset
    
    # Get distributed training info
    rank = int(os.environ.get("RANK", 0))
    world_size = int(os.environ.get("WORLD_SIZE", 1))
    local_rank = int(os.environ.get("LOCAL_RANK", 0))
    
    print(f"Starting training on rank {rank}/{world_size}")
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(local_rank)}")
        torch.cuda.set_device(local_rank)
    
    # Load model and tokenizer
    model_name = "distilbert-base-uncased"
    print(f"Loading model: {model_name}")
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Load and tokenize dataset
    print("Loading IMDB dataset...")
    dataset = load_dataset("imdb", split="train[:1000]")
    
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)
    
    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir="/tmp/output",
        num_train_epochs=1,
        per_device_train_batch_size=8,
        learning_rate=2e-5,
        logging_steps=10,
        save_strategy="no",
        report_to="none",
        ddp_find_unused_parameters=False,
    )
    
    # Train
    trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset)
    trainer.train()
    print(f"Training complete on rank {rank}")

## 4. Configure and submit the training job

In [6]:
# Create TransformersTrainer (progress tracking enabled by default)
trainer = TransformersTrainer(
    func=train_func,
    num_nodes=NUM_NODES,
    resources_per_node={"nvidia.com/gpu": GPUS_PER_NODE},
)

print("Trainer configured")

Trainer configured


In [7]:
# Create client and get runtime
if NAMESPACE:
    backend_config = KubernetesBackendConfig(namespace=NAMESPACE)
    client = TrainerClient(backend_config=backend_config)
else:
    client = TrainerClient()

runtime = client.get_runtime(name="torch-distributed")
print(f"Using runtime: {runtime.name}")

/Users/knema/Library/Python/3.9/lib/python/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.oai-kft-ibm.ibm.rh-ods.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Using runtime: torch-distributed


/Users/knema/Library/Python/3.9/lib/python/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.oai-kft-ibm.ibm.rh-ods.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/knema/Library/Python/3.9/lib/python/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.oai-kft-ibm.ibm.rh-ods.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/knema/Library/Python/3.9/lib/python/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.oai-kft-ibm.ibm.rh-ods.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#t

In [13]:
# Submit the training job
JOB_NAME = client.train(trainer=trainer, runtime=runtime)
print(f"Job submitted: {JOB_NAME}")

/Users/knema/Library/Python/3.9/lib/python/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.oai-kft-ibm.ibm.rh-ods.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Job submitted: t0cf232610b6


## 5. Monitor the training job

You can monitor progress in the OpenShift AI dashboard under Model training.

In [9]:
# Check job status
job = client.get_job(name=JOB_NAME)
print(f"Job: {job.name}")
print(f"Status: {job.status}")

Job: w1b7566893b1
Status: Created


In [10]:
# Wait for job to complete
import time

print(f"Waiting for job to complete...")
while True:
    job = client.get_job(name=JOB_NAME)
    print(f"Status: {job.status}")
    if job.status in ["Complete", "Failed"]:
        break
    time.sleep(15)

print(f"Job finished: {job.status}")

Waiting for job to complete...
Status: Created
Status: Created
Status: Complete
Job finished: Complete


Logs for w1b7566893b1:


<generator object KubernetesBackend.get_job_logs at 0x113d02200>

## 6. Cleanup

In [12]:
# Delete the training job
client.delete_job(name=JOB_NAME)
print(f"Job {JOB_NAME} deleted")

/Users/knema/Library/Python/3.9/lib/python/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.oai-kft-ibm.ibm.rh-ods.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Job w1b7566893b1 deleted


## Summary

In this notebook, you learned how to:

1. Install and configure the Kubeflow SDK
2. Define a distributed training function for HuggingFace Transformers
3. Configure and submit a training job using TransformersTrainer
4. Monitor training progress using the SDK and dashboard

**Next steps:**
- Try larger models and datasets
- Enable JIT checkpointing for fault tolerance
- Configure Kueue for resource scheduling